## Discrete Event Simulation: API and Usage
The class `Wire` and the functions `inverter`, `andGate`, and `orGate` represent a small description language of digital circuits.

We now give the implementation of this class and its functions which allow us to simulate circuits.

These implementations are based on a simple API for discrete event simulation.

A discrete event simulator performs actions, specified by the user at a given moment.

An action is a function that doesn’t take any parameters and which returns `Unit`:
```scala
type Action = () => Unit

```

The time is simulated; it has nothing to with the actual time.